In [1]:
# ALL THE IMPORTS

import keras
from __future__ import print_function
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, Maximum, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, MaxPooling1D, Reshape, GaussianNoise, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import random
import numpy as np
from sklearn.metrics import confusion_matrix
from random import randint
from scipy.ndimage import rotate
import math
from scipy import misc
import scipy.ndimage
from sklearn.metrics import confusion_matrix
from keras import regularizers
from numpy.random import permutation
from keras import metrics
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import sys, glob, argparse
import numpy as np
import math, cv2
from scipy.stats import multivariate_normal
import time
from sklearn import svm
from numpy.testing import assert_allclose
#from nose.tools import raises
#from cyvlfeat.gmm import gmm
#from cyvlfeat.fisher import fisher
from skimage.measure import shannon_entropy
import numpy as np
import matplotlib
import pylidc as pl
import matplotlib.pyplot as plt
from skimage.measure import find_contours
from keras.layers.merge import concatenate
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import RemoteMonitor
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.applications.mobilenet import MobileNet
from scipy.misc import imresize
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from sklearn.model_selection import StratifiedKFold
from scipy.misc import imread
from keras import backend as k
import tensorflow as tf
from scipy.misc import imsave


Using TensorFlow backend.


In [2]:
# FUNCTION TO GET THE BASE MODEL

def get_mobilenet_model():
    base_model  = MobileNet(input_shape=(224,224,3), include_top=False)
    #base_model  = InceptionV3(input_shape=(224,224,3), include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    sgd = optimizers.SGD(lr=0.0001)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')
    #print (model.summary())
    return model

In [3]:
def read_files(files, test_data, test_data_label):
    for f in files:
        t = imread(f)
        t = t/255.0
        test_data.append(t)
        fn = '_'.join(f.split('/')[-1].split('.')[0].split('_')[1:])
        #fn = '_'.join(f.split('/')[-1].split('.')[0].split('_'))
        test_data_label.append(fn)
    return [test_data, test_data_label]

# Function to read test data
def get_test_data(crsd):
    files1 = glob.glob(filepath_a+'fold'+str(crsd)+'/Test/1/*.jpg')
    files2 = glob.glob(filepath_a+'fold'+str(crsd)+'/Test/2/*.jpg')
    test_data = []
    test_data_label = []
    #print(files1)
    test_data, test_data_label = read_files(files1, test_data, test_data_label)
    test_data, test_data_label = read_files(files2, test_data, test_data_label)
    
    #print('test data label ', test_data_label)
    
    malignancy_label_test = []
    file_label    = []
    ml            = []
    orig_filename = []
    
    for idk, t in enumerate(test_data_label):
        label_data = t.split('_')
        pv = t.split('_')[-2]
        temp = [0, 0]
        temp[int(pv)-1] = 1
        malignancy_label_test.append(temp)
        file_label.append(t)
        ml.append(pv)
    
    malignancy_label_test = np.array(malignancy_label_test)
    test_data             = np.array(test_data)
    
    return [test_data, malignancy_label_test, file_label, ml]

In [ ]:
# TRAINING FOR CLASSIFICATION

# Folder structure
# image_sections
#       -- fold1
#           -- Train
#              -- 1
#              -- 2
#           -- Validation
#              -- 1
#              -- 2
#           -- Test
#              -- 1
#              -- 2


ensemble     = 1
init_weights =  None

filesc     = glob.glob('/media/pranjal/part2/FILTERDATANEW_h/*.npy')
filepath_a = '/image_sections/'
batch_size = 32
crsd       = 1

print('FOLD STARTED ', crsd)

# Obtain the base model
model = get_mobilenet_model()

# Create the Data generators for Train and validation splits
train_datagen      = ImageDataGenerator(rescale=1./255, rotation_range=180.0,
                                        horizontal_flip=True, vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255, rotation_range=180.0,
                                        horizontal_flip=True, vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
   filepath_a+'/fold'+str(crsd)+'/Train/',
   batch_size=32,
   target_size=(224, 224))

validation_generator = validation_datagen.flow_from_directory(
   filepath_a+'/fold'+str(crsd)+'/Validation/',
   batch_size=128,
   target_size=(224, 224))

filepath   ='/modelspath/'+str(crsd)+'_mobilenet_weights.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0000001)
er         = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='auto')


# Train the model using fit_generator
model.fit_generator(train_generator,
  steps_per_epoch=400,
  epochs=50,
  validation_data=validation_generator,
  validation_steps=40,
  callbacks=[reduce_lr, checkpoint, er])





In [ ]:
# For predicting using only one image at a time using the above trained model

# Restore the weight with best validation loss
model.load_weights(filepath)

train_data      = []
validation_data = []

test_data, malignancy_label_test, file_label, ml = get_test_data(crsd)

predictions           = model.predict(test_data, batch_size=100)
all_predictions_value = predictions
predictions_prob      = predictions

ruc_score = roc_auc_score(malignancy_label_test, predictions)
cm    = confusion_matrix(np.argmax(predictions, axis=1), np.argmax(malignancy_label_test, axis=1))
tn, fp, fn, tp = cm.ravel()
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)

print("RUC :", ruc_score, "SENSITIVITY :", sensitivity, "SPECIFICITY :", specificity)

In [4]:
def get_file_name(name):
    fn = '_'.join(name.split('/')[-1].split('.')[0].split('_')[1:])
    return fn
#print(get_file_name(files[0]))

In [ ]:
# FOR TAKING MAX POOLING OF FEATURES OF ALL IMAGE SECTIONS FOR ONE NODULE

crsd  = 1
model = get_mobilenet_model()
split = 'Validation'
b     = 32
ensemble = 1

def get_data_from_folder(files, label, start, batch):
    test_data = []
    test_data_label = []
    for j in range(start, start+batch):
    #for j in range((batch)):
        test_data.append(imread(files[j])/255.0)
        #fn = files[j]
        fn = get_file_name(files[j])
        test_data_label.append(fn)
    test_data = np.array(test_data)
    test_data_label = np.array(test_data_label)
    return test_data, test_data_label

def get_features(files1, files2):
    test_data_label = []
    features        = []
    
    #for i in range(len(files1)):
    for i in range(int(len(files1)/128)):
        start = i*128
        test_data_temp, test_data_label_temp = get_data_from_folder(files1, [1, 0], start, 128)
        #print('test_data_temp shape is ', test_data_temp.shape)
        features_temp = intermediate_layer_model.predict(test_data_temp, batch_size=128)
        #print('features_temp 1 ', start)
        for t in features_temp:
            features.append(t)
        for t in test_data_label_temp:
            test_data_label.append(t)
    
    #for i in range(len(files2)):
    for i in range(int(len(files2)/128)):
        start = i*128
        test_data_temp, test_data_label_temp = get_data_from_folder(files2, [0, 1], start, 128)
        features_temp = intermediate_layer_model.predict(test_data_temp, batch_size=128)
        #print('features_temp 2 ', start)
        for t in features_temp:
            features.append(t)
        for t in test_data_label_temp:
            test_data_label.append(t)
    #test_data_label = np.array(test_data_label)
    #features        = np.array(features)
    #print(len(features), len(test_data_label.shape))
    return features, test_data_label

crsd = 1
model.load_weights('/modelspath/'str(crsd)+'_mobilenet_weights.h5')

files1 = glob.glob('/imagesections/fold'+str(crsd)+'/Validation/1/*.jpg')
files2 = glob.glob('/imagesections/fold'+str(crsd)+'/Validation/2/*.jpg')

# Get the output of fully connected layer
layer_name = model.layers[-2].name
intermediate_layer_model  = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
features, test_data_label = get_features(files1, files2)

h = {}
for i, t in enumerate(test_data_label):
    h.setdefault(t, [])
    h[t].append(features[i])
for k in h:
    h[k] = np.max(h[k], axis=0)
features_name = []
features = []
for k in h:
    features_name.append(k)
    features.append(h[k])
features      = np.array(features)
features_name = np.array(features_name)

print('SAVING FEATURES FILE ', crsd)
print(features_name.shape)
print(features.shape)

np.save('/featurespath/fold-'+str(crsd)+'/'+split+'_mobilenet_features.h5',       features)
np.save('/featurespath/fold-'+str(crsd)+'/'+split+'_mobilenet_features_names.h5', features_name)



In [4]:
# TAKES THE ORIGINAL MODEL AS INPUT AND OUTPUTS THE LATER HALF OF THE MODEL

def get_model_pool(model):
    main_input = Input(shape=(1024, ))
    malignancy        = Dense(2, activation='softmax')(main_input)
    pool_model = Model(inputs=[main_input], outputs=[malignancy])
    # SETTING WEIGHTS FOR LATER HALF
    layer_name = model.layers[-1].name
    layer      = model.get_layer(layer_name)
    weights    = layer.get_weights()
    #sgd = optimizers.SGD(lr=0.0001)
    pool_model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam',
                      metrics=[metrics.categorical_accuracy])
    new_layer_name = pool_model.layers[-1].name
    newlayer       = pool_model.get_layer(new_layer_name)
    newlayer.set_weights(weights)
    return pool_model

In [ ]:
# FOR PREDICTION USING MAX POOLING OF FEATURES FOR STAGE 2 TRAINING AND PREDICTION

from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor

crsd  = 1
model = get_mobilenet_model()

split = 'Test'
b     = 32
ensemble  = 1
all_accu  = []
all_accua = []
all_mse = []

values = {}
files  = glob.glob('/media/pranjal/de24af8d-2361-4ea2-a07a-1801b54488d9/FILTERDATANEW_h/*')
for f in files:
    tp = f.split('/')[-1]
    a  = tp.split('.')[0]
    b  = float(tp.split('_')[-1][0:3])
    values[a] = b

crsd = 1
traindata       = np.load('/featurespath/fold-'+str(crsd)+'/'+split+'_mobilenet_features.h5.npy') 
nodulenametrain = np.load('/featurespath/fold-'+str(crsd)+'/'+split+'_mobilenet_features_names.h5.npy')

maxpool    = np.load('/featurespath/fold-'+str(crsd)+'_'+split+'_mobilenet_features.h5.npy')
nodulename = np.load('/featurespath/fold-'+str(crsd)+'_'+split+'_mobilenet_features_names.h5.npy')

h           = {}
final_label = []
train_label = []

for t in nodulename:
    final_label.append(values[t])
for t in nodulenametrain:
    train_label.append(values[t])

train_label = np.array(train_label)
final_label = np.array(final_label)

p = np.random.permutation(len(traindata))
traindata   = traindata[p]
train_label = train_label[p]

#clf = LogisticRegression(solver='liblinear',n_jobs=-1, random_state=0, 
#                    max_iter=155,  class_weight='balanced', intercept_scaling=0.01, C=0.01, tol=0.005)
#clf = SVR(C=1.0, epsilon=0.2, tol=0.0001, gamma=0.0001)
#clf = RandomForestRegressor(max_depth=15, random_state=0)
#clf = SGDRegressor(tol=0.001)

#clf.fit(traindata, train_label)
#clf.fit(traindata, train_label.astype(int))

model.fit([traindata], [train_label], epochs=1000, batch_size=128, 
          shuffle=True, verbose=True, validation_split=0.1, callbacks=[reduce_lr, er, checkpoint])
model.load_weights(filepath)
predictions = model.predict(maxpool)

count     = 0
total     = len(predictions)
incorrect = 0
#y_test    = final_label.astype(int)
y_test    = final_label


In [ ]:
# CODE TO GENERATE TRAINING DATA BY TAKING MULTIPLE CROSS-SECTIONS

import os
import random
import numpy as np
from   sklearn.model_selection import KFold

import scipy.ndimage
import numpy as np
import math
from scipy import misc
import glob
from sklearn.model_selection import StratifiedKFold


count      = 0
checkcount = 0
checked = {}
checked_a = {}
checked_b = {}

files   = glob.glob('/volumes/volume*.npy') 
filessc = []
for f in files:
    at = f.replace('volume-', '')
    filessc.append(at)
files = filessc

alldata = {}

for name in files:
	nodule_id = name.split('.')[0]
	checked[nodule_id]   = name
	checked_a[nodule_id] = name.split('.')[0].split('/')[-1]
	checked_b[nodule_id] = name.split('_')[3]
	alldata[nodule_id]   = []

def rotation_matrix(axis, theta):
    axis = np.asarray(axis)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])
from skimage import filters

def get_empty_classes():
    classes = {}
    classes['1']   = []
    classes['2']   = []
    return classes


keys = list(checked.keys())
random.seed(450)
random.shuffle(keys)

image_size = 50

point = np.array([image_size/2, image_size/2, image_size/2])
temp  = np.zeros((image_size, image_size))
count = 0

divisions = 12
div       = math.pi/divisions
for name in checked:
    a     = np.load(checked[name])
    atx = '/volumes/volume-'+checked[name][len('/volumes/'):]
    a_seg = np.load(atx)
    namep = checked_a[name]
    count = 0
    mode_malignancy = namep.split('_')[-2]
    if mode_malignancy == '2':
        divisions = 7
    else:
        divisions = 5
    div = math.pi/divisions
    print(name)
    
    angles_array = [[0, 0], [math.pi/2, 0], [math.pi/2, math.pi/2]]
    #angles_array = [[0, 0] ]
    
    #for divb in range(0, divisions + 1):
    for abc in range(1):
        for abc_i, abc_j in enumerate(angles_array):
        #for diva in range(0, divisions + 1):
            #anglea = diva * div
            #angleb = divb * div
            anglea = angles_array[abc_i][0]
            angleb = angles_array[abc_i][1]
            
            x = math.cos(anglea) * math.sin(angleb)
            y = math.sin(anglea) * math.sin(angleb)
            z = math.cos(angleb)
            normal = np.array([x, y, z])
            for (i, val) in enumerate(normal):
                if abs(normal[i]) < 0.000001:
                    normal[i] = 0
            (yy, zz) = np.meshgrid(range(image_size), range(image_size))
            yy = yy - image_size/2
            zz = zz - image_size/2
            xx = temp
            rt1 = rotation_matrix([0, 0, 1], anglea)
            a1 = np.reshape(xx, -1)
            a2 = np.reshape(yy, -1)
            a3 = np.reshape(zz, -1)
            val1 = np.array([a1, a2, a3])
            val1_prime = np.dot(rt1, val1)
            new_axis   = np.dot([0, 1, 0], rt1)
            mag = np.sqrt(new_axis.dot(new_axis))
            new_axis = new_axis / mag
            rt2 = rotation_matrix(new_axis, angleb)
            val2_prime = np.dot(rt2, val1_prime)
            val2_prime = val2_prime + image_size/2
            sp = scipy.ndimage.map_coordinates(a, val2_prime, order=3, mode='nearest')
            sp = np.reshape(sp, [image_size, image_size])
            #sp = imresize(sp, (224, 224))
            #sp = sp.astype(float)
            #sp = sp/255.0
            temp_data = [namep, sp]
            alldata[name].append(temp_data)
            count = count + 1